# Import the required modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from numpy.lib.recfunctions import append_fields
from function import *
#Use seaborn style
plt.style.use('seaborn')

## Import the required files

In [ ]:
enrolment1920 = np.genfromtxt('./datasets/Enrolment201920.csv',delimiter=',',skip_header=15, names=True,usecols=(0,1,2,3,4,5,6,7),dtype=['U50' for i in range(7)]+[ 'i8'])

In [ ]:
#Filter out those that have extra commas and additional strings in the numbers
def transformCommasUni(line):
    out = ''
    insideQuote = False
    outQuote = False
    num = [str(i) for i in range(0,10)]
    c = 0
    #as long as the number is less than the length of line
    while c < len(line):
        #" inside represents that theres is a string occuring
        if line[c] == '"':
            insideQuote = not insideQuote
        #if the string occurs in a number we skip the quotation mark
        if line[c] == '"'  and line[c-1] in num :
            out = out[:] + line[c+1]
            
            c+=1
        #When it is a number in thousands remove the strings and quote
        elif insideQuote == True and line[c] in num  and line[c+1] == ',':
            if c+4 < len(line):
                out = out[:-1] + line[c] +line[c+2] +line[c+3]+line[c+4]
            c+= 4
        #When it is a number in ten thousands remove the strings and quote "10,"
        elif insideQuote == True and line[c] in num  and line[c+2] == ',':
            if c+5 < len(line):
                out = out[:-1] + line[c] +line[c+1] +line[c+3]+line[c+4] + line[c+5]
            c+= 5
        #Replace comma with . for words 
        elif insideQuote == True and line[c] == ',' and line[c+1] not in num:
            out += '.'
        
        else:
            out += line[c]
        c+= 1
    return out
nums = []
# Open the file
f = open("./datasets/enrolmentSGUni.csv", "r")
replaced = (transformCommasUni(line) for line in f)
#Import the file
enrolmentSGUni = np.genfromtxt(replaced,delimiter=',',names=True,  dtype=[('year','i8'),('sex','U50'),('course','U50'),('intake','i8'),('enrolment','i8'),('graduate','i8')])
f.close()
# Open the file
g = open('./datasets/enrolmentSGPoly.csv','r')
replaced1 = (transformCommasUni(line) for line in g)
#Import the file
enrolmentSGPoly = np.genfromtxt(replaced1,delimiter=',',names=True,  dtype=[('year','i8'),('sex','U50'),('course','U50'),('intake','i8'),('enrolment','i8'),('graduate','i8')])
g.close()

In [ ]:
#Overall data analysis for singapore data
class Singapore:
    def __init__(self):
        pass
    def runALL(self,n,j,level):
        for i in n:
            year = set(i['year'])
            print(f'\nAnalysis of the {year} {j} rate of Singapore in {level}')
            #Filter out the years and print out the average std of each years
            #Filtering each datasets
            print(functionNeed.characteristics(i[i['sex'] == 'MF'][j]))
sg = Singapore()

In [ ]:
#Filtering out the data needed before analysing and finding the nature of each column to reduce the need to check the whole of datasets
total2005 = enrolmentSGUni[(enrolmentSGUni['year'] == 2005) & (enrolmentSGUni['sex'] == 'MF')]
total2020 =enrolmentSGUni[(enrolmentSGUni['year'] == 2020) & (enrolmentSGUni['sex'] == 'MF')]
combine = [total2005,total2020]
#Printing out the unique values of the first dataset
print('*** Enrolment to University (Singapore) on students base on years and by Course ***\n')
#Run the output for both years
for i in combine:
    columns =  len(i)
    print(f'There are {i.shape[0]} rows in this filtered dataset and {columns} columns')
    print(f'\nThe names of the columns are:')
    functionNeed.columnInArr(i.dtype.names,i)
    functionNeed.uniqueValues(i.dtype.names,i)


*** Enrolment to University (Singapore) on students base on years and by Course ***

There are 15 rows in this filtered dataset and 15 columns

The names of the columns are:
- year <class 'numpy.dtype[int64]'> isnumeric: True, null values: False 
- sex <class 'numpy.dtype[str_]'> isnumeric: False, null values: False 
- course <class 'numpy.dtype[str_]'> isnumeric: False, null values: False 
- intake <class 'numpy.dtype[int64]'> isnumeric: True, null values: False 
- enrolment <class 'numpy.dtype[int64]'> isnumeric: True, null values: False 
- graduates <class 'numpy.dtype[int64]'> isnumeric: True, null values: False 

The unique values of each column are:
1 unique values in year
1 unique values in sex
15 unique values in course
15 unique values in intake
15 unique values in enrolment
14 unique values in graduates
There are 15 rows in this filtered dataset and 15 columns

The names of the columns are:
- year <class 'numpy.dtype[int64]'> isnumeric: True, null values: False 
- sex <class 

# Change the courses to field

## Singaproe

In [ ]:
#Categorise the courses into field
def changeToField(n):
    courses = np.unique(n['course'])
    courses = np.delete(courses,len(courses)-2)
    field = []
    field_map = {
    'Accountancy': 'Business',
    '"Architecture. Building & Real Estate"': 'Arts',
    'Business & Administration': 'Business',
    'Dentistry': 'Medicine',
    'Education': 'Education',
    'Engineering Sciences': 'Engineering',
    'Fine & Applied Arts': 'Arts',
    'Health Sciences': 'Medicine',
    'Humanities & Social Sciences': 'Arts',
    'Information Technology': 'Computing',
    'Law': 'Law',
    'Mass Communication': 'Business',
    'Medicine': 'Medicine',
    '"Natural. Physical & Mathematical Sciences"': 'Science',\
    "Natural & Mathematical Sciences":'Science',
    'Services': 'Services',
    "Applied Arts":"Arts"}
    for i in (n['course']):
        for key,value in field_map.items():
            if i == key:
                field.append(value)
    return field


In [ ]:
#Appending the field to the datasets
enrolmentSGUni = append_fields(enrolmentSGUni, 'Field', changeToField(enrolmentSGUni))

In [ ]:
#Appending the field to the datasets
enrolmentSGPoly = append_fields(enrolmentSGPoly, 'Field', changeToField(enrolmentSGPoly))


# Data analysis

In [ ]:
#Run all analysis
sg.runALL(combine,'intake','Uni')


Analysis of the {2005} intake rate of Singapore in Uni
The number of elements is:              14
The min is:               0
The max is:            4028
The mean is:          833.87
The variance is:      1180656.25
The median is         233.000
The standard deviation is:        1086.580
The lower quartile is         167.500
The upper quartile is        1210.500

(0, 4028, 833.8666666666667, 1180656.2488888886, 1086.580070169193, 14, 233.0, 1210.5, 167.5)

Analysis of the {2020} intake rate of Singapore in Uni
The number of elements is:              15
The min is:              72
The max is:            5225
The mean is:         1398.40
The variance is:      2190482.37
The median is         483.000
The standard deviation is:        1480.028
The lower quartile is         333.000
The upper quartile is        2271.000

(72, 5225, 1398.4, 2190482.373333333, 1480.0278285671973, 15, 483.0, 2271.0, 333.0)


In [ ]:
#Run all analysis
sg.runALL(combine,'graduates','Uni')


Analysis of the {2005} graduates rate of Singapore in Uni
The number of elements is:              13
The min is:               0
The max is:            3859
The mean is:          668.73
The variance is:       981839.26
The median is         187.000
The standard deviation is:         990.878
The lower quartile is          59.500
The upper quartile is         920.500

(0, 3859, 668.7333333333333, 981839.2622222219, 990.8780259054199, 13, 187.0, 920.5, 59.5)

Analysis of the {2020} graduates rate of Singapore in Uni
The number of elements is:              15
The min is:              51
The max is:            4621
The mean is:         1168.93
The variance is:      1604484.33
The median is         429.000
The standard deviation is:        1266.682
The lower quartile is         302.500
The upper quartile is        1669.500

(51, 4621, 1168.9333333333334, 1604484.3288888892, 1266.682410428474, 15, 429.0, 1669.5, 302.5)


In [ ]:
total2005P = enrolmentSGPoly[(enrolmentSGPoly['year'] == 2005) & (enrolmentSGPoly['sex'] == 'MF')]
total2020P =enrolmentSGPoly[(enrolmentSGPoly['year'] == 2020) & (enrolmentSGPoly['sex'] == 'MF')]
combineP = [total2005P,total2020P]
#Printing out the unique values of the first dataset
print('***Intake Enrolment and Graduate for Poly (Singapore) on students base on years and by Course ***\n')
#Running both years at the same time
for i in combineP:
    columns =  len(i)
    print(f'There are {i.shape[0]} rows in this filtered dataset and {columns} columns')
    print(f'\nThe names of the columns are:')
    functionNeed.columnInArr(i.dtype.names,enrolmentSGPoly)
    functionNeed.uniqueValues(i.dtype.names,enrolmentSGPoly)

***Intake Enrolment and Graduate for Poly (Singapore) on students base on years and by Course ***

There are 12 rows in this filtered dataset and 12 columns

The names of the columns are:
- year <class 'numpy.dtype[int64]'> isnumeric: True, null values: False 
- sex <class 'numpy.dtype[str_]'> isnumeric: False, null values: False 
- course <class 'numpy.dtype[str_]'> isnumeric: False, null values: False 
- intake <class 'numpy.dtype[int64]'> isnumeric: True, null values: False 
- enrolment <class 'numpy.dtype[int64]'> isnumeric: True, null values: False 
- graduates <class 'numpy.dtype[int64]'> isnumeric: True, null values: False 
- Field <class 'numpy.dtype[str_]'> isnumeric: False, null values: False 

The unique values of each column are:
16 unique values in year
2 unique values in sex
13 unique values in course
347 unique values in intake
371 unique values in enrolment
345 unique values in graduates
9 unique values in Field
There are 12 rows in this filtered dataset and 12 columns


In [ ]:
#Run all for poly
sg.runALL(combineP,'intake','Poly')


Analysis of the {2005} intake rate of Singapore in Poly
The number of elements is:              12
The min is:              81
The max is:            7826
The mean is:         1742.17
The variance is:      4992565.47
The median is         821.500
The standard deviation is:        2234.405
The lower quartile is         176.750
The upper quartile is        2100.000

(81, 7826, 1742.1666666666667, 4992565.472222223, 2234.4049481287457, 12, 821.5, 2100.0, 176.75)

Analysis of the {2020} intake rate of Singapore in Poly
The number of elements is:              12
The min is:             108
The max is:            5919
The mean is:         1751.17
The variance is:      2855936.14
The median is         950.000
The standard deviation is:        1689.952
The lower quartile is         605.250
The upper quartile is        2517.000

(108, 5919, 1751.1666666666667, 2855936.138888888, 1689.9515196859606, 12, 950.0, 2517.0, 605.25)


C:\Users\ruizh\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:755: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


In [ ]:
#Run all for poly graduates
sg.runALL(combineP,'graduates','Poly')


Analysis of the {2005} graduates rate of Singapore in Poly
The number of elements is:              11
The min is:               0
The max is:            6536
The mean is:         1376.25
The variance is:      3571502.85
The median is         487.500
The standard deviation is:        1889.842
The lower quartile is         168.750
The upper quartile is        1523.000

(0, 6536, 1376.25, 3571502.8541666665, 1889.8420183091143, 11, 487.5, 1523.0, 168.75)

Analysis of the {2020} graduates rate of Singapore in Poly
The number of elements is:              12
The min is:              93
The max is:            6327
The mean is:         1855.00
The variance is:      3270840.00
The median is        1104.500
The standard deviation is:        1808.546
The lower quartile is         629.750
The upper quartile is        2433.750

(93, 6327, 1855.0, 3270840.0, 1808.5463776193299, 12, 1104.5, 2433.75, 629.75)


## Function for changing into field

In [ ]:
#Getting the total number by field
def field(n,cat):
    fieldlUnique = np.unique(n['Field'])
    years = np.unique(n['year'])
    coursesInfo = {
    'Business':[],
    'Arts':[],
    'Business':[],
    'Medicine':[],
    'Education':[],
    'Engineering':[],
    'Arts':[],
    'Medicine':[],
    'Arts':[],
    'Computing':[],
    'Law':[],
    'Business':[],
    'Medicine':[],
    'Science':[],
    'Services':[]}
    for j in range(len(years)):
        for i in range(len(fieldlUnique)):
            
            data = n[(n['Field'] == fieldlUnique[i]) & (n['year'] == years[j])&(n['sex'] == 'MF') ][cat]
            coursesInfo[fieldlUnique[i]].append(sum(data))
    return coursesInfo

# Comparing the intake of poly and uni by field

In [ ]:
#Getting the years and the unique values
years = np.unique(enrolmentSGPoly['year'])
courseinfoUniI = field(enrolmentSGUni,'intake')
#Sorting 
courseinfoUniI = dict(sorted(courseinfoUniI.items(), key=lambda x:x[1]))
courseinfoPolyI = field(enrolmentSGPoly,'intake')
#Sorting
courseinfoPolyI = dict(sorted(courseinfoPolyI.items(), key=lambda x:x[1]))

In [ ]:
%matplotlib qt
#Create a subplot
fig,ax = plt.subplots(3,2,figsize=(20,18))
fig.set_facecolor("lightblue")
fig.suptitle('Comparing the intake of polytechnic and university by field', fontsize=30,color='navy')
line = ax[0,0]
line.set_title('University intake from 2005 to 2020')
colors = ['lightskyblue','seagreen','#d956c6','darkolivegreen','#ac91e5','#8e49fc','navy','goldenrod','brown']
#Creating a line plot
lineBox = [
    line.plot(years,i[1],label=i[0],color = j)
    for i,j in zip(courseinfoUniI.items(),colors)
]
line.set_ylim(0,8500)
#Creating the heatmap
heatmap = ax[1,0]
heatmap.set_title('Showing the density of University intake')
im =heatmap.imshow(courseinfoUniI.values(), cmap='viridis')
heatmap.set_aspect(aspect=0.6)
line.legend(loc='upper left')
heatmap.set_xticks(np.arange(len(years)), labels= np.arange(5,21,1))
heatmap.set_yticks(np.arange(len(courseinfoUniI.items())), labels=courseinfoUniI.keys())
heatmap.grid(False)

fig.colorbar(im,ax=heatmap)

#Creating the buttons for interactivity
button = ax[2,0]
param_x = [0.2]
param_y = [0.6]
OptionButton = button.plot([0.2, 0.8],np.full((2,1), param_y[0]),linewidth=40,solid_capstyle='round',color = 'navy',label = 'Show data on boxplots')
optionMarkers = button.plot(param_x[0],param_y[0],label='Show data on boxplots',marker='o',color='b',linestyle='None')
button.text(param_x[0] + 0.08,param_y[0],'Show data on boxplots',verticalalignment='center',label='Show data on boxplots',color='White')
button.set_xticks([])
button.set_yticks([])
button.set_xlim(0, 1)
button.set_ylim(0, 1)
button.set_title("Option for boxplot")
button.set_facecolor('lightblue')
paramsData = {"Show data on boxplots":False}

#Set the poly comparison
line2 = ax[0,1]
#Creating a line plot for poly
lineBox2 = [
    line2.plot(years,i[1],label=i[0],color = j)
    for i,j in zip(courseinfoPolyI.items(),colors)
]
line2.legend(loc='upper left')
line2.set_title('Polytechnic intake from 2005 to 2020')
#Creating the heatmap for poly
heatmap2 = ax[1,1]
heatmap2.set_title('Showing the density of Polytechnic intake')
im2 =heatmap2.imshow(courseinfoPolyI.values(), cmap='viridis')
heatmap2.set_aspect(aspect=0.6)
heatmap2.set_xticks(np.arange(len(years)), labels= np.arange(5,21,1))
heatmap2.set_yticks(np.arange(len(courseinfoPolyI.items())), labels=courseinfoPolyI.keys())
heatmap2.grid(False)
fig.colorbar(im2,ax=heatmap2)

#Creating the buttons for interactivity for poly
button2 = ax[2,1]
button2.set_facecolor('lightblue')
param_x1 = [0.2,0.2]
param_y1 = [0.6,0.4]
OptionButton2 = button2.plot([0.2, 0.8],np.full((2,1), param_y[0]),linewidth=40,solid_capstyle='round',color = 'navy',label = 'Show data on boxplots')
optionMarkers2 = button2.plot(param_x1[0],param_y1[0],label='Show data on boxplots',marker='o',color='b',linestyle='None')

button2.text(param_x1[0] + 0.08,param_y1[0],'Show data on boxplots',verticalalignment='center',label='Show data on boxplots',color='White')

button2.set_xlim(0, 1)
button2.set_ylim(0, 1)
button2.set_xticks([])
button2.set_yticks([])
button2.set_title("Option for boxplot")
paramsData2= {"Show data on boxplots":False}

#Contains point ffunction to see if the user click on the buttons
def containsPoint(line, x, y):
    firstPoint, secondPoint = line.get_xydata()
    if firstPoint[1]-0.15 <= y and y <= secondPoint[1] + 0.15:
        if firstPoint[0] <= x and x <= secondPoint[0]:
            return True
    return False

#The onclick event and response
def onclick(event):
    if (event.inaxes == button):
        box = OptionButton[0]
        #getting the option box
        #Checking if press contains the points
        if (containsPoint(box, event.xdata, event.ydata)):
            paramsData[box.get_label()] = not paramsData[box.get_label()]
        marker = optionMarkers[0]
        
        if (paramsData[marker.get_label()]):
            #If click on show data on boxplots button, graph will switch
            if (marker.get_label() == 'Show data on boxplots'):
                line.clear()
                line.set_title('Boxplot for the spread of intake for each field (University)')
                line.boxplot(courseinfoUniI.values(),labels=courseinfoUniI.keys())
                [button.plot(param_x[0],param_y[0],label='Show data on boxplots',marker='o',color='g',linestyle='None')]
                line.set_ylim(0,8500)
                plt.draw()
                plt.show()
        else:
             #If click on show data on boxplots button, graph will switch
            if (marker.get_label() == 'Show data on boxplots'):
                line.clear()
                lineBox = [
                    line.plot(years,i[1],label=i[0],color = j)
                    for i,j in zip(courseinfoUniI.items(),colors)
                ]
                line.set_title('University intake from 2005 to 2020')
                line.legend(loc='upper left')
                [button.plot(param_x[0],param_y[0],label='Show data on boxplots',marker='o',color='b',linestyle='None')]
                line.legend()
                line.set_ylim(0,8500)
                plt.draw()
                plt.show()
    elif (event.inaxes == button2):
        box = OptionButton2[0]
        if (containsPoint(box, event.xdata, event.ydata)):
            paramsData2[box.get_label()] = not paramsData2[box.get_label()]
        marker = optionMarkers2[0]
        

        if (paramsData2[marker.get_label()]):
            #If click on show data on boxplots button, graph will switch
            if (marker.get_label() == 'Show data on boxplots'):
                line2.clear()
                line2.boxplot(courseinfoPolyI.values(),labels=courseinfoPolyI.keys())
                [ button2.plot(param_x1[0],param_y1[0],label='Show data on boxplots',marker='o',color='g',linestyle='None')]
                line2.set_title('Boxplot for the spread of intake for each field (Polytechnic)')
                plt.draw()
                plt.show()
        else:
            #If click on show data on boxplots button, graph will switch
            if (marker.get_label() == 'Show data on boxplots'):
                line2.clear()
                lineBox2 = [
                    line2.plot(years,i[1],label=i[0],color = j)
                    for i,j in zip(courseinfoPolyI.items(),colors)
                ]
                line2.legend(loc='upper left')
                [ button2.plot(param_x1[0],param_y1[0],label='Show data on boxplots',marker='o',color='b',linestyle='None')]
                line2.set_title('Polytechnic intake from 2005 to 2020')
                line2.legend()
                plt.draw()
                plt.show()
                   
                

plt.show()
cid = fig.canvas.mpl_connect('button_press_event', onclick)
plt.show()


C:\Users\ruizh\AppData\Local\Temp\ipykernel_37564\4103944792.py:25: MatplotlibDeprecationWarning: Auto-removal of grids by pcolor() and pcolormesh() is deprecated since 3.5 and will be removed two minor releases later; please call grid(False) first.
  fig.colorbar(im,ax=heatmap)
C:\Users\ruizh\AppData\Local\Temp\ipykernel_37564\4103944792.py:59: MatplotlibDeprecationWarning: Auto-removal of grids by pcolor() and pcolormesh() is deprecated since 3.5 and will be removed two minor releases later; please call grid(False) first.
  fig.colorbar(im2,ax=heatmap2)


# Compare the intake and graduate for 2005 and 2020 for both uni and poly

In [ ]:
courseinfoUniG = field(enrolmentSGUni,'graduates')
#sorting
courseinfoUniG = dict(sorted(courseinfoUniG.items(), key=lambda x:x[1]))
courseinfoPolyG = field(enrolmentSGPoly,'graduates')
#Sorting
courseinfoPolyG = dict(sorted(courseinfoPolyG.items(), key=lambda x:x[1]))

In [ ]:
#Setting the colors and font
colors = ['lightskyblue','seagreen','#d956c6','darkolivegreen','#ac91e5','#8e49fc','navy','goldenrod','brown']
font = {'fontname':'Roboto'}

In [ ]:

def PieChartSG(cat,types):
    %matplotlib qt
    fig, ax = plt.subplots(2, 2, figsize=(15, 15))
    fig.suptitle(f'Intake and Graduate of {types}',fontsize=30,color='navy')
    fig.set_facecolor("lightgray")
    #converting it to list
    value = list(cat.values())
    #Getting the 2005 values
    valueNeeded = [i[0] for i in value]
    #Getting the ratio
    ratio = [i/sum(valueNeeded) for i in valueNeeded]
    #constructiong the pie chart
    wedges, autotexts = ax[0, 0].pie(valueNeeded,textprops=dict(color="w",fontsize=9),colors = colors,explode=(0,0,0,0,0,0,0,0,0.1), shadow=True,labels = cat.keys())
    #Setting the font colour to black
    [autotexts[i].set_color('black') for i in  range(len(autotexts))]
    ax[0, 0].set_title('Proportion of intake by course 2005',**font)
    ax[0, 0].legend(wedges, cat.keys(),title="Subject of Study",loc="center right")
    ax[0, 0].legend_.set_bbox_to_anchor((-0, 0.5))
    ratio.sort(reverse=True)
    valueNeeded.sort(reverse=True)
    label = []
    #Getting the top course labels 
    for i in range(0, 5):
        for key, values in cat.items():
            if valueNeeded[i] == values[0]:
                label.append(key)
    bottom = 1
    width = .2

    # Adding from the top matches the legend.
    for j, (height, label) in enumerate(reversed([*zip(ratio, label)])):
        bottom -= height
        #Setting the hieght of the bar
        bc = ax[0, 1].bar(0, height, width, bottom=bottom, color='C0', label=label,
                          alpha=0.1 + 0.1 * j)
        ax[0, 1].bar_label(bc, labels=[f"{height:.2%}"], label_type='center')

    ax[0, 1].set_title('Percentage of Highest 5 intake courses',**font)
    ax[0, 1].axis('off')
    ax[0, 1].set_xlim(- 2.5 * width, 2.5 * width)
    ax[0, 1].legend()
    ax[0, 1].legend_.set_bbox_to_anchor((0.2, 0.8))
    #Getting 2020 values
    valueNeeded1 = [i[-1] for i in value]

    ratio1 = [i/sum(valueNeeded1) for i in valueNeeded1]

    wedges, autotexts = ax[1, 0].pie(valueNeeded1,
                                     textprops=dict(color="w",fontsize=9),colors = colors,explode=(0,0,0,0,0,0,0,0,0.1), shadow=True,labels=cat.keys())
    [autotexts[i].set_color('black') for i in  range(len(autotexts))]

    ax[1, 0].set_title('Proportion of intake by course 2020',**font)
    ax[1, 0].legend(wedges, cat.keys(),title="Subject of Study",loc="center right")
    ax[1, 0].legend_.set_bbox_to_anchor((-0, 0.5))
    label = []
    ratio1.sort(reverse=True)
    valueNeeded1.sort(reverse=True)
    for i in range(0, 5):
        for key, values in cat.items():
            if valueNeeded1[i] == values[-1]:
                label.append(key)
    bottom = 1
    width = .2

    # Adding from the top matches the legend.
    for j, (height, label) in enumerate(reversed([*zip(ratio1, label)])):
        bottom -= height
        bc = ax[1, 1].bar(0, height, width, bottom=bottom, color='C0', label=label,
                          alpha=0.1 + 0.1 * j)
        ax[1, 1].bar_label(bc, labels=[f"{height:.2%}"], label_type='center')

    ax[1, 1].set_title('Percentage of Highest 5 intake courses',**font)
    ax[1, 1].axis('off')
    ax[1, 1].set_xlim(- 2.5 * width, 2.5 * width)
    ax[1, 1].legend()
    ax[1, 1].legend_.set_bbox_to_anchor((0.2, 0.8))

    #Setting the button for clicking
    buttons = plt.axes([0.86, 0.8, 0.13, 0.02])
    param_x = [0.2]
    param_y = [0.6]
    OptionButton = buttons.plot([0.2, 1.2],np.full((2, 1), param_y[0]),linewidth=15,solid_capstyle='round',color=(0, 0, 0, 0.1),label='Show graduates 2005')
    optionMarkers = buttons.plot(param_x[0],param_y[0],label='Show graduates 2005',marker='o',color='b',linestyle='None') 
    buttons.set_facecolor("lightgray")
    buttons.text(param_x[0] + 0.08, param_y[0],'Show graduates 2005',verticalalignment='center',label='Show graduates 2005')
    buttons.set_xticks([])
    buttons.set_yticks([])

    buttons.set_xlim(0, 1)
    buttons.set_ylim(0, 1)

    buttons.set_title("Option for boxplot")
    paramsData = {"Show graduates 2005": False}
# For 2020
#Setting button for click
    buttons2 = plt.axes([0.86, 0.2, 0.13, 0.02])
    buttons2.set_facecolor("lightgray")
    param_x1 = [0.2]
    param_y1 = [0.6]
    OptionButton2 = buttons2.plot([0.2, 1.2],np.full((2, 1), param_y1[0]),linewidth=15,solid_capstyle='round',color=(0, 0, 0, 0.05),label='Show graduates 2020')
    optionMarkers2 = buttons2.plot(param_x1[0],param_y1[0],label='Show graduates 2020',marker='o',color='b',linestyle='None')
    buttons.set_facecolor("lightgray")
    buttons2.text(param_x1[0] + 0.08, param_y1[0],'Show graduates 2020',verticalalignment='center',label='Show graduates 2020')
    buttons2.set_xticks([])
    buttons2.set_yticks([])

    buttons2.set_xlim(0, 1)
    buttons2.set_ylim(0, 1)

    buttons2.set_title("Option for boxplot" )
    paramsData2 = {"Show graduates 2020": False}
    #Update the graph functions
    def update(n,year,x,cat,index,color,option):
        ax[x,0].clear()
        ax[x,1].clear()
        value = list(n.values())
        valueNeeded = [i[index] for i in value]
        ratio = [i/sum(valueNeeded) for i in valueNeeded]

        wedges, autotexts = ax[x, 0].pie(valueNeeded,textprops=dict(color="w"),colors = colors, explode=(0,0,0,0,0,0,0,0,0.1),shadow=True,labels=n.keys())
        [autotexts[i].set_color('black') for i in  range(len(autotexts))]
        ax[x, 0].set_title(f'Proportion of {cat} by course {year}')
        ax[x, 0].legend(wedges, n.keys(),title="Subject of Study",loc="center right",)
        ax[x, 0].legend_.set_bbox_to_anchor((0, 0.5))
        ratio.sort(reverse=True)
        valueNeeded.sort(reverse=True)
        label = []
        for i in range(0, 5):
            for key, values in n.items():
                if valueNeeded[i] == values[index]:
                    label.append(key)
        bottom = 1
        width = .2
        # Adding from the top matches the legend.
        for j, (height, label) in enumerate(reversed([*zip(ratio, label)])):
            bottom -= height
            bc = ax[x, 1].bar(0, height, width, bottom=bottom, color='C0', label=label,
                            alpha=0.1 + 0.1 * j)
            ax[x, 1].bar_label(bc, labels=[f"{height:.2%}"], label_type='center')
        ax[x, 1].set_title(f'Percentage of Highest 5 graduates courses')
        ax[x, 1].axis('off')
        ax[x, 1].set_xlim(- 2.5 * width, 2.5 * width)
        ax[x, 1].legend()
        ax[x, 1].legend_.set_bbox_to_anchor((0.2, 0.8))
        [option.plot(param_x1[0],param_y1[0],label='Show graduates 2020',marker='o',color=color,linestyle='None')]
        plt.draw()
        plt.show()
    #Create the clicking event 
    def onclick(event):
        #if axes equals to first button
        if (event.inaxes == buttons):
            box = OptionButton[0]
            if (containsPoint(box, event.xdata, event.ydata)):
                paramsData[box.get_label(
                )] = not paramsData[box.get_label()]
            
            marker = optionMarkers[0]
            if (paramsData[marker.get_label()]):
                if types == 'Poly':
                    c = courseinfoPolyG
                else:
                    c = courseinfoUniG
                if (marker.get_label() == 'Show graduates 2005'):
                    update(c,2005,0,'graduates',0,'g',buttons)
            else:
                if marker.get_label() == 'Show graduates 2005':
                    if types == 'Poly':
                        c = courseinfoPolyI
                        update(c,2005,0,'intake',0,'b',buttons)
                    else:
                        c = courseinfoUniI
                        update(c,2005,0,'intake',0,'b',buttons)
                    
        elif (event.inaxes == buttons2):
              #if axes equals to second button
            clicker = OptionButton2[0]
            if (containsPoint(clicker, event.xdata, event.ydata)):
                paramsData2[clicker.get_label()] = not paramsData2[clicker.get_label()]
            marker = optionMarkers2[0]
            if (paramsData2[marker.get_label()]):
                if types == 'Poly':
                    c = courseinfoPolyG
                else:
                    c = courseinfoUniG
                if (marker.get_label() == 'Show graduates 2020'):
                    update(c,2020,1,'graduates',-1,'g',buttons2)

            else:
                if marker.get_label() == 'Show graduates 2020':
                    if types == 'Poly':
                        c = courseinfoPolyI
                        update(c,2020,1,'intake',-1,'b',buttons2)
                    else:
                        c = courseinfoUniI
                        update(c,2020,1,'intake',-1,'b',buttons2)            
    fig.canvas.mpl_connect('button_press_event', onclick)
    plt.show()


In [ ]:
#Running the infomation for poly
PieChartSG(courseinfoPolyI,'Poly')

In [ ]:
#Running the infomation for University
PieChartSG(courseinfoUniI,'Uni')

## Comparison between Singaproe and UK intake and enrolment 2020

In [ ]:
#Function for UK datasets
class Uk:
    def __init__(self):
        pass
    def data(self,n):
        for i,j in n.items():

            print(f'\nThere are {j.shape[0]} rows in this filtered intake dataset and {columns} columns')
            print(f'\nThe names of the columns are:')
            functionNeed.columnInArr(j.dtype.names,j)
            functionNeed.uniqueValues(j.dtype.names,j)
    def runALL(self,n,category):
        for i,j in n.items():
            year = '20'+i[-2:]
            print(f'\nAnalysis of the {year} {category} rate of UK')
            #Filter out the years and print out the average std of each years
            #Filtering each datasets
            print(functionNeed.characteristics(j['Number']))
uk = Uk()

In [ ]:
filterReq =(enrolment1920['Number'] != 0) & (enrolment1920['CAH_level_1'] != 'Total')&(enrolment1920['CAH_level_1'] != 'Total science CAH level 1')&(enrolment1920['CAH_level_1'] != 'Total non-science CAH level 1')&(enrolment1920['Academic_Year'] == '2019/20') & (enrolment1920['Sex'] == 'Total') & (enrolment1920['Country_of_HE_provider'] == 'All') & (enrolment1920['Mode_of_study']=='Full-time') & (enrolment1920['Level_of_study'] == 'All undergraduate') & (enrolment1920['First_year_marker'] == 'First year')
intakeUK = {"intake20":[]}
intakeUK['intake20'] = enrolment1920[filterReq]


In [ ]:
uk.data(intakeUK)


There are 21 rows in this filtered intake dataset and 12 columns

The names of the columns are:
- CAH_level_1 <class 'numpy.dtype[str_]'> isnumeric: False, null values: False 
- First_year_marker <class 'numpy.dtype[str_]'> isnumeric: False, null values: False 
- Level_of_study <class 'numpy.dtype[str_]'> isnumeric: False, null values: False 
- Mode_of_study <class 'numpy.dtype[str_]'> isnumeric: False, null values: False 
- Country_of_HE_provider <class 'numpy.dtype[str_]'> isnumeric: False, null values: False 
- Sex <class 'numpy.dtype[str_]'> isnumeric: False, null values: False 
- Academic_Year <class 'numpy.dtype[str_]'> isnumeric: False, null values: False 
- Number <class 'numpy.dtype[int64]'> isnumeric: True, null values: False 

The unique values of each column are:
21 unique values in CAH_level_1
1 unique values in First_year_marker
1 unique values in Level_of_study
1 unique values in Mode_of_study
1 unique values in Country_of_HE_provider
1 unique values in Sex
1 unique val

In [ ]:
#Run all functions to analysis
uk.runALL(intakeUK,'intake')


Analysis of the 2020 intake rate of UK
The number of elements is:              21
The min is:            1625
The max is:          109965
The mean is:        26733.81
The variance is:    754277237.87
The median is       15905.000
The standard deviation is:       27464.108
The lower quartile is        4860.000
The upper quartile is       32200.000

(1625, 109965, 26733.809523809523, 754277237.8684807, 27464.10817537101, 21, 15905.0, 32200.0, 4860.0)


In [ ]:
#Relate the UK courses to Singapore
def filterWithRelation(n,category):
    relation_course = []
    for i in range(0,len(category[n]['CAH_level_1'])):
        if 'medicine' in category[n]['CAH_level_1'][i].lower():
            relation_course.append('Medicine')
        elif 'veterinary' in category[n]['CAH_level_1'][i].lower():
            relation_course.append(category[n]['CAH_level_1'][i][2:])
        elif 'combined' in category[n]['CAH_level_1'][i].lower():
            relation_course.append(category[n]['CAH_level_1'][i][2:])
        elif 'physical' in category[n]['CAH_level_1'][i].lower() or 'mathematical' in category[n]['CAH_level_1'][i].lower():
            relation_course.append('Physics and Mathematics Sciences')
        elif  'language' in category[n]['CAH_level_1'][i].lower() or 'biological' in category[n]['CAH_level_1'][i].lower() or 'general' in category[n]['CAH_level_1'][i].lower() or 'psychology' in category[n]['CAH_level_1'][i].lower() or 'geographical' in category[n]['CAH_level_1'][i].lower() or  'humanities' in category[n]['CAH_level_1'][i].lower() or  'langauge' in category[n]['CAH_level_1'][i].lower() or  'social' in category[n]['CAH_level_1'][i].lower():
            relation_course.append('Humanities and Social science')
        elif 'engineering'in category[n]['CAH_level_1'][i].lower():
            relation_course.append(category[n]['CAH_level_1'][i][2:])
        elif 'computing' in category[n]['CAH_level_1'][i].lower():
            relation_course.append(category[n]['CAH_level_1'][i][2:])
        elif 'law' in category[n]['CAH_level_1'][i].lower():
            relation_course.append(category[n]['CAH_level_1'][i][2:])
        elif 'business' in category[n]['CAH_level_1'][i].lower():
            relation_course.append(category[n]['CAH_level_1'][i][2:])
        elif 'communication' in category[n]['CAH_level_1'][i].lower():
            relation_course.append(category[n]['CAH_level_1'][i][2:])
        elif 'creative' in category[n]['CAH_level_1'][i].lower():
            relation_course.append(category[n]['CAH_level_1'][i][2:])
        elif 'education' in category[n]['CAH_level_1'][i].lower():
            relation_course.append(category[n]['CAH_level_1'][i][2:])
        elif 'combined' in category[n]['CAH_level_1'][i].lower():
            print('cobime')
            relation_course.append(category[n]['CAH_level_1'][i][2:])
        
    return (relation_course)


In [ ]:
#Append relational course column
intakeUK['intake20'] = append_fields(intakeUK['intake20'], 'Relational_Course', filterWithRelation('intake20',intakeUK)
,'U50')

In [ ]:
#Related the relational course to field
courses = np.unique(intakeUK['intake20']['Relational_Course'])

fieldUK = []
field_map = {
'Business': 'Business',
'"Architecture_Building & Real Estate"': 'Arts',
'Business & Administration': 'Business',
'Dentistry': 'Medicine',
'Education': 'Education',
' Engineering and technology': 'Engineering',
'Fine & Applied Arts': 'Arts',
'Health Sciences': 'Medicine',
'Humanities and Social science': 'Arts',
'Computing': 'Computing',
'Law': 'Law',
'Communication': 'Business',
'Medicine': 'Medicine',
'Physics and Mathematics Sciences': 'Science',
"Natural & Mathematical Sciences":'Science',
'Services': 'Services',
"Veterinary sciences":"Science",
"Creative arts and design":"Arts",
"Combined and general studies":"General Studies"}
for i in (intakeUK['intake20']['Relational_Course']):
    for key,value in field_map.items():
        if key in i:
            fieldUK.append(value)

In [ ]:
#Append the field column for comparison
intakeUK['intake20'] = append_fields(intakeUK['intake20'], 'Field',fieldUK)

In [ ]:
#Getting the total number of intake by field UK
def fieldUkData(n,cat):
    fieldlUnique = np.unique(n['Field'])
    years = np.unique(n['Academic_Year'])
    coursesInfo = {
    'Business':[],
    'Arts':[],
    'Business':[],
    'Medicine':[],
    'Education':[],
    'Engineering':[],
    'Arts':[],
    'Medicine':[],
    'Arts':[],
    'Computing':[],
    'Law':[],
    'Business':[],
    'Medicine':[],
    'Science':[],
    "General Studies":[]}
    for j in range(len(years)):
        for i in range(len(fieldlUnique)):
            
            data = n[(n['Field'] == fieldlUnique[i])][cat]
            coursesInfo[fieldlUnique[i]].append(sum(data))
    return coursesInfo

In [ ]:
courseUKUniI=fieldUkData(intakeUK['intake20'],'Number')
#Sorting
courseUKUniI = dict(sorted(courseUKUniI.items(), key=lambda x:x[1]))

In [ ]:
def PieChartCombined(n,sg,uk):
    %matplotlib qt
    fig,ax = plt.subplots(2,2,figsize=(20,30))
    fig.suptitle(f'{n.capitalize()} comparsion between Singapore and UK',fontsize=30,color='navy')
    fig.set_facecolor('lightgray')
    #Getting the values to list
    value = list(uk.values())
    #Getting the year 2020
    valueNeeded = [i[0] for i in value]
    #Getting the ratio
    ratio = [i/sum(valueNeeded) for i in valueNeeded]
    
   #Implementing the pie chart
    wedges, autotexts = ax[0,0].pie(valueNeeded,
                                textprops=dict(color="w"),colors=colors,explode=(0,0,0,0,0,0,0,0,0.1),shadow=True,labels=uk.keys())
    [autotexts[i].set_color('black') for i in  range(len(autotexts))]
    ax[0,0].set_title('Proportion of intake by course UK 2020',**font)
    ax[0,0].legend(wedges, uk.keys(),title="Subject of Study",loc="center right" )
    ax[0,0].legend_.set_bbox_to_anchor((-0,0.5))
    ratio.sort(reverse=True)
    valueNeeded.sort(reverse=True)
    label = []
    #Getting the top 5 labels 
    for i in range(0,5):
        for key,values in uk.items():
            if valueNeeded[i] == values[0]:
                label.append(key)
    bottom = 1
    width = .2

    # Adding from the top matches the legend.
    for j, (height, label) in enumerate(reversed([*zip(ratio, label)])):
        bottom -= height
        bc = ax[0,1].bar(0, height, width, bottom=bottom, color='C0', label=label,
                    alpha=0.1 + 0.1 * j)
        ax[0,1].bar_label(bc, labels=[f"{height:.0%}"], label_type='center')

    ax[0,1].set_title('Percentage of Highest 5 intake courses',**font)
    ax[0,1].axis('off')
    ax[0,1].set_xlim(- 2.5 * width, 2.5 * width)
    ax[0,1].legend()
    ax[0,1].legend_.set_bbox_to_anchor((0.2,0.8))
    #Getting Sg values
    value = list(sg.values())
    #Getting the year 2020
    valueNeeded1 = [i[-1] for i in value]
    #Getting the ratio
    ratio1 = [i/sum(valueNeeded1) for i in valueNeeded1]
    #Implmenting the pie chart
    wedges, autotexts = ax[1, 0].pie(valueNeeded1,
                                    textprops=dict(color="w"), colors=colors,labels=sg.keys(),shadow=True,explode=(0,0,0,0,0,0,0,0,0.1))
    [autotexts[i].set_color('black') for i in  range(len(autotexts))]
    ax[1, 0].set_title('Proportion of intake by course SG 2020',**font)
    ax[1, 0].legend(wedges, sg.keys(),title="Subject of Study",loc="center right",)
    ax[1, 0].legend_.set_bbox_to_anchor((-0, 0.5))
    label = []
    ratio1.sort(reverse=True)
    valueNeeded1.sort(reverse=True)
    for i in range(0, 5):
        for key, values in sg.items():
            if valueNeeded1[i] == values[-1]:
                label.append(key)
    bottom = 1
    width = .2

    # Adding from the top matches the legend.
    for j, (height, label) in enumerate(reversed([*zip(ratio1, label)])):
        bottom -= height
        bc = ax[1, 1].bar(0, height, width, bottom=bottom, color='C0', label=label,
                        alpha=0.1 + 0.1 * j)
        ax[1, 1].bar_label(bc, labels=[f"{height:.0%}"], label_type='center')

    ax[1, 1].set_title('Percentage of Highest 5 intake courses',**font)
    ax[1, 1].axis('off')
    ax[1, 1].set_xlim(- 2.5 * width, 2.5 * width)
    ax[1, 1].legend()
    ax[1, 1].legend_.set_bbox_to_anchor((0.2, 0.8))
    plt.show()


In [ ]:
#Creating the pie chart
PieChartCombined('intake',courseinfoUniI,courseUKUniI)

findfont: Font family ['Roboto'] not found. Falling back to DejaVu Sans.
